## Hit-Rate
Evaluation at different ranking positions showing the proportion of DrugMechDB ground truth edges captured at various thresholds.

In [1]:
from src.cbr_trainer.query_dataloader import DataLoader
from src.cbr_trainer.rgcn_model import HeteroRGCN
from src.cbr_trainer.rgcn_model import HeteroRGCNLayer
from src.important_paths.explainer import MaskExplainer
from src.cbr_trainer.utils.dist_scr import L2Dist
from collections import *

import pandas as pd
import os
import torch
from tqdm import tqdm
import torch.nn as nn
import numpy as np
import dgl
import torch.nn.functional as F

In [2]:

data_dir = "data/MIND/"
data_name = "MIND"
paths_file_dir = "MIND_cbr_subgraph_knn-15_branch-1000.pkl"
device =  "cuda"

In [3]:
# Initialize data loader
dataset_obj = DataLoader(data_dir, 
                         data_name, 
                         paths_file_dir)

05/01/2025 09:21:47 - INFO - root - Loading and processing graph data...
05/01/2025 09:22:13 - INFO - root - Creating node mappings...
05/01/2025 09:22:13 - INFO - root - Loading paths data...
05/01/2025 09:22:14 - INFO - root - Processing CSV data...
05/01/2025 09:22:15 - INFO - root - Processing path data...
100%|█████████| 878/878 [00:01<00:00, 786.13it/s]
05/01/2025 09:22:18 - INFO - root - Building node labels...
05/01/2025 09:22:18 - INFO - root - Building train dataset...
100%|██████████| 760/760 [06:32<00:00,  1.94it/s]
05/01/2025 09:28:51 - INFO - root - Building test dataset...
100%|██████████| 243/243 [02:33<00:00,  1.58it/s]
05/01/2025 09:31:25 - INFO - root - Building dev dataset...
100%|██████████| 116/116 [01:12<00:00,  1.60it/s]


In [4]:
model_path = 'link_prediction_results/MIND/model/'
model_name = 'best_model_MIND_021624.pth'

model_dir = os.path.join(model_path, model_name)


In [5]:
model = torch.load(model_dir, map_location=device)
model.to(device)
model.eval()

/tmp/ipykernel_1916608/3725338474.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_dir, map_location=device)


HeteroRGCN(
  (emb): HeteroEmbedding(
    (embeds): ModuleDict(
      (AnatomicalEntity): Embedding(3577, 128)
      (BiologicalProcessOrActivity): Embedding(21062, 128)
      (ChemicalSubstance): Embedding(40739, 128)
      (Disease): Embedding(13945, 128)
      (MacromolecularMachine): Embedding(128294, 128)
      (OrganismTaxon): Embedding(9762, 128)
      (Pathway): Embedding(5363, 128)
      (PhenotypicFeature): Embedding(5428, 128)
    )
  )
  (layer1): HeteroRGCNLayer(
    (weight0): Linear(in_features=128, out_features=128, bias=True)
    (weight): ModuleDict(
      (associated_with_AawD): Linear(in_features=128, out_features=128, bias=True)
      (associated_with_AawP): Linear(in_features=128, out_features=128, bias=True)
      (site_of_AsoD): Linear(in_features=128, out_features=128, bias=True)
      (site_of_AsoPW): Linear(in_features=128, out_features=128, bias=True)
      (affects_BPafD): Linear(in_features=128, out_features=128, bias=True)
      (associated_with_BPawD): L

In [6]:
from types import SimpleNamespace
args_dict = {
"lr_" :0.1,
"num_epochs": 10,
"penalty" : 1.0,
"degree_thr" :10,
"split": "test",
"alpha": 1.0,
"beta": 1.0}
args = SimpleNamespace(**args_dict)


In [7]:
nodes_mapping = dataset_obj.nodes_mapping
if args.split == "test":
        drug_dis_train = [(item["seed_entities"][0], ans) for item in dataset_obj.raw_test_data for ans in item["answer"]]
    

In [8]:
# Initialize explainer
explainer = MaskExplainer(model, device, args, args.split)

In [9]:
#Load true indications from DMDB

dmdb_paths= pd.read_pickle("data/DMDB_ground-truth_paths.pkl")
                           
dmdb_pairs = set()
for drug_id,dis_id in zip(dmdb_paths["mrn_drug"], dmdb_paths["mrn_dis"]):
    dmdb_pairs.add((drug_id,dis_id ))

Evaluation

In [12]:
hit_ranking = defaultdict(list)
total_paths = defaultdict(list)

In [14]:
for drug_id, dis_id in tqdm(dmdb_pairs):
    try:
        orig_pred, ml_ghomo, ml_eweight_dict, ml_ghomo_eweights, ntype_hetero_nids_to_homo_nids, nn_graph, nn_idx, paths = explainer.explain(drug_id, 
                                                                                                       dis_id, 
                                                                                                       dataset_obj, 
                                                                                                      True, 
                                                                                                        args.split)
    
    except:
        continue 
        
    #extract seuqnce of nodes
    list_src_paths = []
    for i in range(len(paths)):

        src_path = []
        for path in paths[i]:
            src, edge, tgt = path[0], path[1], path[2]
            src_path.append(src)
        src_path.append(dis_id)
        list_src_paths.append(src_path)
    
    #DMDB paths
    true_paths = dmdb_paths.query("mrn_drug == @drug_id & mrn_dis == @dis_id")['path_mrn'].to_list()

    
    #Evaluate ranking
    total_paths[(drug_id, dis_id)].append(len(paths))
    for i in range(len(list_src_paths)):
        if list_src_paths[i] in true_paths:
            hit_ranking[(drug_id, dis_id)].append(i)
            print(i, list_src_paths[i])

  1%|▏            | 1/72 [00:09<11:30,  9.72s/it]

34 ['CHEBI:31548', 'NCBIGene:5739', 'GO:0006954', 'DOID:4483']



  3%|▎            | 2/72 [00:18<10:54,  9.35s/it]

168 ['CHEBI:5439', 'NCBIGene:2554', 'GO:1904862', 'DOID:50433']



  4%|▌            | 3/72 [00:30<11:50, 10.29s/it]

...

7 ['CHEBI:64354', 'NCBIGene:6559', 'GO:0055078', 'DOID:10763']



 97%|███████████▋| 70/72 [12:17<00:19,  9.95s/it]

5 ['CHEBI:16359', 'NCBIGene:1593', 'DOID:4810']



100%|████████████| 72/72 [12:40<00:00, 10.57s/it]

152 ['CHEBI:8502', 'NCBIGene:7173', 'GO:0006590', 'DOID:7998']


Evaluate Results

In [16]:
pair_list =[]
rank_list =[]
total = []

for pair in (hit_ranking): 
    for rank in (hit_ranking[pair]):
        if rank == "NA":
            continue
        pair_list.append(pair)
        rank_list.append(rank +1)
        total.append(total_paths[pair][0])
path_ranking = pd.DataFrame()
path_ranking["pair"] =pair_list
path_ranking["rank"] =rank_list
path_ranking["total_paths"] =total

In [17]:
#calculate MRR
rr_results = []
for rank_item in path_ranking["rank"]:
    rr_results.append(1/rank_item)
print("MRR: ", np.sum(rr_results) / len(rr_results))

MRR:  0.15254399611871386


In [21]:
hits = [10, 50, 100, 150]

In [22]:
for hit in hits:
    result = len([i for i in path_ranking['rank'] if i <= hit]) /len(rr_results)
    print(hit, result)

10 0.4090909090909091
50 0.7159090909090909
100 0.7840909090909091
150 0.8863636363636364
